# Общие указания

Аналогично ДЗ 1

каждое задание - 2 балла, доп задания - 1 балл. Всего 9 - потом приводится к 10.

In [1]:
import numpy as np
import time
import pandas as pd
import networkx as nx
from grakel import graph_from_networkx
from grakel.kernels import ShortestPath, GraphletSampling, WeisfeilerLehman, NeighborhoodHash, PyramidMatch, OddSth, CoreFramework,  Propagation, RandomWalk, SvmTheta, VertexHistogram
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KernelDensity
from sklearn.metrics.pairwise import pairwise_distances
from scipy.optimize import minimize
import random
from types import SimpleNamespace
from ucimlrepo import fetch_ucirepo 
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern
from scipy.stats import norm


import warnings
warnings.filterwarnings('ignore')

# Задание 1.

Реализуйте непараметрический LDA (лекция 2, слайд 36). Возьмите датасеты из предыдущего ДЗ (3 задание) (если не делали - подберите или сгенерируйте) и попытайтесь побить затюненный регулеризованный LDA (если не делали предыдущее задание или ваша реализация получилась неудачной, то можете взять реализацию из sklearn) подбирая kernel (перебирайте популярные, а также попробуйте придумать свой kernel), lambda (можно подбирать константу, а можно - функцию, лекция 2 - слайд 26). Сравните время работы алгоритмов.

**Дополнительно**: реализуйте также local likelihood logistic regression (слайды 27 и 33 из второй лекции в помощь) и сравните с моделями из основной части.

Возьмем RDA из прошлого дз

In [2]:
class RegularizedLDA:
    def __init__(self):
        self.class_covariances = None
        self.class_covariances_inv = None
        self.discriminant_constants = None
        self.class_means = None
        self.class_probabilities = None
        self.num_classes = None

    def fit(self, X, y, alpha=0.5, gamma=0.1):
        classes = np.unique(y)
        n_samples, n_features = X.shape
        class_counts = {cl: np.sum(y == cl) for cl in classes}
        n_classes = np.array(list(class_counts.values()))

        # вероятности классов (π_k) 
        self.class_probabilities = n_classes / n_samples

        self.num_classes = len(classes)
        self.class_means = np.array([X[y == cl].mean(axis=0) for cl in classes]) # средние значения для каждого класса
        global_covariance = np.cov(X, rowvar=False)   # общая ковариационная матрица
        global_covariance = gamma * global_covariance + (1 - gamma) * np.std(X, axis=0)[:, None] * np.eye(n_features)  # регуляризация ков-й матрицы

        # ковариации внутри классов с регуляризацией
        self.class_covariances = []
        self.class_covariances_inv = []
        for cl in classes:
            class_cov = np.cov(X[y == cl], rowvar=False)
            regularized_cov = alpha * class_cov + (1 - alpha) * global_covariance
            self.class_covariances.append(regularized_cov)
            self.class_covariances_inv.append(np.linalg.inv(regularized_cov))

       
        self.discriminant_constants = [
            (-0.5 * np.sum(np.log(np.linalg.eigvals(self.class_covariances[i])))+ np.log(self.class_probabilities[i]))
            for i in range(self.num_classes)
        ]  # конст дискриминантных функций (для каждого класса)

    def predict(self, X):
        discriminants = np.zeros((X.shape[0], self.num_classes))
        for i in range(self.num_classes):
            diff = X - self.class_means[i]  # разность между объектом и средним для каждого класса
            #  дискриминантная функция для каждого класса
            discriminants[:, i] = np.sum(diff @ self.class_covariances_inv[i] * diff, axis=1)
            discriminants[:, i] = -0.5 * discriminants[:, i] + self.discriminant_constants[i]

        return np.argmax(discriminants, axis=1)

    def accuracy(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return np.mean(y_pred == y_test)


### Реализуем класс - непараметрический LDA с ядрами, доступными KernelDensity и попробуем добавить несколько кастомных

In [3]:
class NonParametricLDA:
    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.class_kdes = None  # ядерные плотностные оценки для каждого класса
        self.kernel = kernel
        self.class_probabilities = None  # вероятности классов
        self.num_classes = None  # количество классов
        self.custom_kernels = {
            'custom': self._custom_kernel,
            'logg': self._log_kernel,  # логарифмическое ядро
            'custom2': self._custom2_kernel,  # гиперболическое ядро
            'sigmoid': self._sigmoid_kernel  
        }
    

    def fit(self, X, y):
        classes = np.unique(y)
        n_samples, n_features = X.shape
        class_counts = {cl: np.sum(y == cl) for cl in classes}

        self.class_probabilities = np.array([class_counts[cl] / n_samples for cl in classes])  # вероятности классов
        self.num_classes = len(classes)

        # ядерные плотностные модели для каждого класса
        self.class_kdes = {}
        for cl in classes:
            X_class = X[y == cl]
            kde = self._fit_kde(X_class)
            self.class_kdes[cl] = kde

    def predict(self, X):
        log_densities = np.zeros((X.shape[0], self.num_classes))
        for i, cl in enumerate(self.class_kdes.keys()):
            kde = self.class_kdes[cl]
            # получаем плотности для каждого объекта 
            log_densities[:, i] = kde(X) 

        log_densities += np.log(self.class_probabilities)
        return np.argmax(log_densities, axis=1)  # класс с наибольшей вероятностью

    def accuracy(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return np.mean(y_pred == y_test)

    def _fit_kde(self, X_class):
        # если ядро доступно KernelDensity
        if self.kernel in ['gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine']:
            def kdee(X):
                kde = KernelDensity(kernel=self.kernel, bandwidth=self.bandwidth)
                kde.fit(X_class)
                return kde.score_samples(X)
            return kdee
        # если кастомное ядро
        else:
            def kde_(X):
                dist = pairwise_distances(X, X_class, metric='euclidean')
                kernel_values = self.custom_kernels[self.kernel](dist, self.bandwidth)
                return np.sum(kernel_values, axis=1) / len(X_class)
            return kde_

    def _custom_kernel(self, dist, bandwidth):
        return np.cos(dist / (2 * bandwidth)) * (dist <= bandwidth)
    
    def _log_kernel(self, dist, bandwidth):
        return np.log(1 + dist / bandwidth) * (dist <= bandwidth)
    
    def _custom2_kernel(self, dist, bandwidth):
        return 1 / (dist / bandwidth) ** 2 + 1

    def _sigmoid_kernel(self, dist, bandwidth):
        return np.tanh(dist / bandwidth)



Загрузим датасеты из прошлого дз

In [4]:
def load_scale():
    df = pd.read_csv("https://archive.ics.uci.edu/static/public/12/data.csv")
    dataset = {}
    dataset['data'] = df.drop(columns='class')  
    dataset['target'] = df['class']  
    label_encoder = LabelEncoder()
    dataset['target'] = label_encoder.fit_transform(dataset['target'])

    return dataset

def load_seeds():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt"
    df = pd.read_csv(url, sep="\\t+", header=None)
    
    return df



### Посмотрим, какой алгоритм работает быстрее и у какого выше accuracy

LDA возьмем и из прошлого дз, и из sklearn

In [5]:
# список датасетов UCI 
datasets = {
    'iris': load_iris(),
    'wine': load_wine(),
    'breast_cancer': load_breast_cancer(),
    'seeds': load_seeds(),
    "balance_scale": load_scale(),

}


def compare_rda_logreg():
    results = []
    time_results = []

    for dataset_name, dataset in datasets.items():
        if isinstance(dataset, dict):
            X = dataset['data']
            y = dataset['target']
        else:
            X = dataset.iloc[:, :-1].values
            y = dataset.iloc[:, -1].values - 1
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

        scaler = StandardScaler()  # стандартизируем данные
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        start_time = time.time()
        param_grid_rda = {
            'alpha': [0.1, 0.5, 1.0],
            'gamma': [0.01, 0.1, 0.5, 1.0]
        }
        best_rda_sc = 0
        best_rda_params = None
        
        # попробуем разные значения гиперпараметров alpha и gamma для RDA
        for alpha in param_grid_rda['alpha']:
            for gamma in param_grid_rda['gamma']:
                rda_model = RegularizedLDA()
                rda_model.fit(X_train_scaled, y_train, alpha=alpha, gamma=gamma)
                rda_sc = rda_model.accuracy(X_test_scaled, y_test)
                
                if rda_sc > best_rda_sc:
                    best_rda_sc = rda_sc
                    best_rda_params = {'alpha': alpha, 'gamma': gamma}
        rda_time = time.time() - start_time
        

        #подбираем параметры для log reg
        start_time = time.time()
        param_grid_lr = {'C': [0.01, 0.1, 1, 10, 100]} 
        lr_model = LogisticRegression(max_iter=10000)
        grid_search_lr = GridSearchCV(lr_model, param_grid_lr, cv=5, n_jobs=-1)
        grid_search_lr.fit(X_train_scaled, y_train)
        lr_best_model = grid_search_lr.best_estimator_
        lr_score = lr_best_model.score(X_test_scaled, y_test)
        lr_time = time.time() - start_time
        
        # подбираем ядро и лямбду для NonParametricLDA 
        start_time = time.time()
        kernels = ['gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine', 'custom', 'logg', 'custom2', 'sigmoid' ]
        bandwidth = [0.1, 0.5, 1.0, 2.0]  # лямбды (для ширины)
        
        best_nlda_score = 0
        best_kernel = None
        best_bandwidth = 1.0
        
        for kernel in kernels:
            for b in bandwidth:
                n_model = NonParametricLDA(kernel=kernel, bandwidth=b)
                n_model.fit(X_train_scaled, y_train)
                score = n_model.accuracy(X_test_scaled, y_test)
                # print(kernel,score)
                if score > best_nlda_score:
                    best_nlda_score = score
                    best_kernel = kernel
                    best_bandwidth = b
        nlda_time = time.time() - start_time

    
        
        # LDA из sklearn
        start_time = time.time()
        param_grid_rda_sklearn = {'shrinkage': ['auto', 0.1, 0.5, 1.0]}  # регуляризация с shrinkage
        rda_sklearn_model = LDA(solver='eigen')
        grid_search_rda = GridSearchCV(rda_sklearn_model, param_grid_rda_sklearn, cv=5, n_jobs=-1)
        grid_search_rda.fit(X_train_scaled, y_train)
        rda_sklearn_best_model = grid_search_rda.best_estimator_
        rda_sklearn_score = rda_sklearn_best_model.score(X_test_scaled, y_test)
        rda_sklearn_time = time.time() - start_time


        results.append({
            'Dataset': dataset_name,
            'Best RDA accuracy': best_rda_sc,
            'Best RDA Alpha': best_rda_params['alpha'],
            'Best RDA Gamma': best_rda_params['gamma'],
            'Best Logistic Regression accuracy': lr_score,
            'Best Logistic Regression C': grid_search_lr.best_params_['C'],
            'LDA (sklearn) accuracy': rda_sklearn_score,
            'Best Nonparametric LDA accuracy': best_nlda_score,
            'Best Nonparametric LDA Kernel': best_kernel,
            'Best Nonparam LDA Bandwidth (lamda)': best_bandwidth,
        })
        time_results.append({
            'Dataset': dataset_name,
            'RDA time': rda_time,
            'Logistic Regression time': lr_time,
            'Nonparametric LDA time': nlda_time,
            'LDA (sklearn) time': rda_sklearn_time,
        })

   
    df_res = pd.DataFrame(results)
    df_time = pd.DataFrame(time_results)
    
    return df_res, df_time

results_df, time_df = compare_rda_logreg()
results_df


,Dataset,Best RDA accuracy,Best RDA Alpha,Best RDA Gamma,Best Logistic Regression accuracy,Best Logistic Regression C,LDA (sklearn) accuracy,Best Nonparametric LDA accuracy,Best Nonparametric LDA Kernel,Best Nonparam LDA Bandwidth (lamda)
0,iris,0.933333,1.0,0.01,0.933333,10.0,0.933333,0.933333,tophat,1.0
1,wine,1.000000,1.0,0.01,1.000000,0.1,1.000000,1.000000,gaussian,0.1
2,breast_cancer,0.964912,0.5,0.01,0.973684,1.0,0.973684,0.964912,gaussian,1.0
3,seeds,0.952381,0.1,0.01,0.952381,10.0,0.928571,0.976190,gaussian,0.1
4,balance_scale,0.912000,1.0,0.01,0.912000,100.0,0.880000,0.896000,gaussian,1.0


 - На большинстве датасетов все модели, показывают хорошую точность
 - В датасете "wine"  все методы достигают 100% точности (может указывать на относительную простоту задачи для выбранных классификаторов), на "iris"  все методы достигают 93% точности 
 - В некоторых случаях RDA уступает в точности Logistic Regression и LDA. Например, на "breast_cancer" точность RDA составляет 96.49%, тогда как Logistic Regression и LDA достигают 97.37%. На "seeds" точность RDA ниже точности Nonparametric LDA (но на "balance_scale" точность RDA выше точности Nonparametric LDA ).
 
 

In [6]:
time_df

,Dataset,RDA time,Logistic Regression time,Nonparametric LDA time,LDA (sklearn) time
0,iris,0.017347,0.955469,0.019546,0.028530
1,wine,0.018518,0.050628,0.020353,0.035660
2,breast_cancer,0.035210,0.152933,0.126083,0.098787
3,seeds,0.006652,0.042592,0.018643,0.021502
4,balance_scale,0.003567,0.031609,0.060223,0.029641



**Получаем, что logistic Regression и	Nonparametric LDA (и то, и то с подбором параметров) работает дольше, чем rda и lda из sklearn**

## Дополнительно
- реализуйте также local likelihood logistic regression (слайды 27 и 33 из второй лекции в помощь) и сравните с моделями из основной части.

In [7]:
class LLLR:
    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.kernel = kernel
        self.kernels = {
            'gaussian': self._gaussian_kernel,
            'epanechnikov': self._epanechnikov_kernel,
            'tophat': self._tophat_kernel
        }

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.scaler = StandardScaler()
        self.X_train_scaled = self.scaler.fit_transform(X)
    
    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        predictions = []
        
        for x_0 in X_scaled: # для каждого х считаем вес и предсказываем класс
            weights = self._compute_kernel_weights(x_0)  # веса для ядра
            model = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter=10000)
            model.fit(self.X_train_scaled, self.y_train, sample_weight=weights)  #лог рег с учетом весов 
            predictions.append(model.predict(x_0.reshape(1, -1))[0])        
        return np.array(predictions)

    def _compute_kernel_weights(self, x_0):
        # считаем веса для ядра для одного объекта x_0 
        dist = pairwise_distances(self.X_train_scaled , [x_0], metric='euclidean').flatten()
        kernel_values = self.kernels[self.kernel](dist, self.bandwidth)
        return kernel_values

    def _gaussian_kernel(self, dist, bandwidth):
        return np.exp(-dist**2 / (2 * bandwidth**2))

    def _epanechnikov_kernel(self, dist, bandwidth):
        return 3/4 * (1 - (dist / bandwidth)**2) * (np.abs(dist) <= bandwidth)

    def _tophat_kernel(self, dist, bandwidth):
        return (np.abs(dist) <= bandwidth).astype(float)

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)
    


In [8]:
def evaluate_lllr():
    datasets = {
        'iris': load_iris(),
        'wine': load_wine(),
        'breast_cancer': load_breast_cancer(),
        'seeds':load_seeds(),
        'balance_scale': load_scale(),
    }

    results = []
    time_results = []
   
    for dataset_name, dataset in datasets.items():
        if isinstance(dataset, dict):
            X = dataset['data']
            y = dataset['target']
        else:
            X = dataset.iloc[:, :-1].values
            y = dataset.iloc[:, -1].values - 1
            
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
        
        start_time = time.time()
        
        bandwidth = [0.1, 0.5, 1.0, 2.0]
        best_llr_acc = 0
        best_bandwidth = 1.0
        for b in bandwidth:
            model = LLLR(bandwidth=b, kernel='gaussian')
            model.fit(X_train, y_train)
            acc = model.accuracy(X_test, y_test)
            if acc > best_llr_acc:
                best_llr_acc = acc
                best_bandwidth = b
                
        lllr_time = time.time() - start_time
        
        results.append({
            'Dataset': dataset_name,
            'LLLR accuracy': acc,
            'Best LLLR Bandwidth (lamda)': best_bandwidth,
            
           
        })
        time_results.append({
            'Dataset': dataset_name,
            'LLLR time': lllr_time
        })
        
    results_df = pd.DataFrame(results)
    time_df = pd.DataFrame(time_results)
    
    return results_df, time_df

results_lllr, time_results_lllr = evaluate_lllr()
results_lllr


,Dataset,LLLR accuracy,Best LLLR Bandwidth (lamda)
0,iris,0.900000,0.5
1,wine,1.000000,1.0
2,breast_cancer,0.973684,2.0
3,seeds,0.976190,0.5
4,balance_scale,0.896000,1.0


In [9]:
pd.merge(results_df, results_lllr, on='Dataset', how='outer')

,Dataset,Best RDA accuracy,Best RDA Alpha,Best RDA Gamma,Best Logistic Regression accuracy,Best Logistic Regression C,LDA (sklearn) accuracy,Best Nonparametric LDA accuracy,Best Nonparametric LDA Kernel,Best Nonparam LDA Bandwidth (lamda),LLLR accuracy,Best LLLR Bandwidth (lamda)
0,iris,0.933333,1.0,0.01,0.933333,10.0,0.933333,0.933333,tophat,1.0,0.900000,0.5
1,wine,1.000000,1.0,0.01,1.000000,0.1,1.000000,1.000000,gaussian,0.1,1.000000,1.0
2,breast_cancer,0.964912,0.5,0.01,0.973684,1.0,0.973684,0.964912,gaussian,1.0,0.973684,2.0
3,seeds,0.952381,0.1,0.01,0.952381,10.0,0.928571,0.976190,gaussian,0.1,0.976190,0.5
4,balance_scale,0.912000,1.0,0.01,0.912000,100.0,0.880000,0.896000,gaussian,1.0,0.896000,1.0


In [10]:
pd.merge(time_df, time_results_lllr, on='Dataset', how='outer')

,Dataset,RDA time,Logistic Regression time,Nonparametric LDA time,LDA (sklearn) time,LLLR time
0,iris,0.017347,0.955469,0.019546,0.028530,0.260669
1,wine,0.018518,0.050628,0.020353,0.035660,0.217797
2,breast_cancer,0.035210,0.152933,0.126083,0.098787,2.412582
3,seeds,0.006652,0.042592,0.018643,0.021502,0.478018
4,balance_scale,0.003567,0.031609,0.060223,0.029641,0.755712


Получаем, что LLLR работает в среднем (с подбором лямбды) дольше, чем остальные алгоритмы, и результаты:
- на 3/5 датасетов есть алгоритм с таким же результатом (и этот результат-лучший) (wine,breast_cancer,seeds)
- на остальных 2/5 существует алгоритм с результатом лучше

# Задание 2.

Используйте kernel trick для классификации графов с помощью ridge регрессии (она используется для регрессии, но здесь мы сделаем вид, что все ок и будем обрубать значения меньше 0 и больше 1) и svm (в Sklearn можно использовать кастомные kernels).

Датасеты отсюда: https://github.com/FilippoMB/Benchmark_dataset_for_graph_classification

Ядра можно взять отсюда: https://github.com/ysig/GraKeL

Подберите лучшее ядро для всех случаев.

**Дополнительно**: поэкспериментируйте с kernel construction на основе kernels из библиотеки и попробуйте предложить свой kernel, который работает лучше

In [13]:
import  grakel.kernels
print(dir(grakel.kernels))

['CoreFramework', 'EdgeHistogram', 'GraphHopper', 'GraphletSampling', 'HadamardCode', 'Kernel', 'LovaszTheta', 'MultiscaleLaplacian', 'NeighborhoodHash', 'NeighborhoodSubgraphPairwiseDistance', 'OddSth', 'Propagation', 'PropagationAttr', 'PyramidMatch', 'RandomWalk', 'RandomWalkLabeled', 'ShortestPath', 'ShortestPathAttr', 'SubgraphMatching', 'SvmTheta', 'VertexHistogram', 'WeisfeilerLehman', 'WeisfeilerLehmanOptimalAssignment', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_c_functions', '_isomorphism', 'core_framework', 'edge_histogram', 'graph_hopper', 'graphlet_sampling', 'hadamard_code', 'kernel', 'lovasz_theta', 'multiscale_laplacian', 'neighborhood_hash', 'neighborhood_subgraph_pairwise_distance', 'odd_sth', 'propagation', 'pyramid_match', 'random_walk', 'shortest_path', 'subgraph_matching', 'svm_theta', 'vertex_histogram', 'weisfeiler_lehman', 'weisfeiler_lehman_optimal_assignment']


In [14]:
np.random.seed(5)

In [15]:
# конвертируем графы в формат nx
def create_networkx_graph(A, X):
    G = nx.from_numpy_array(A)
    x_tuple = tuple(map(tuple, X))
    nx.set_node_attributes(G, dict(enumerate(x_tuple)), 'features')
    return G

In [16]:
def load_datasets_with_graphs(dataset_name): # options: {easy_small, easy, hard_small, hard}
    # dataset_name = "hard_small" 
    loaded = np.load(f'datasets/{dataset_name}.npz', allow_pickle=True)

    A_train = list(loaded['tr_adj'])  # матрицы смежности для обучающей выборки
    X_train = loaded['tr_feat']  # признаки узлов для обучающей выборки
    y_train = loaded['tr_class']  # классы для обучающей выборки
    # с test аналогично
    A_test = list(loaded['te_adj'])  
    X_test = loaded['te_feat']  
    y_test = loaded['te_class']  
    return A_train, X_train, y_train, A_test, X_test, y_test

Смотреть на accuracy будем в виде процентов (accuracy*100) для наглядности

In [17]:
def kernel_trick_(dataset_name):
    A_train, X_train, y_train, A_test, X_test, y_test  = load_datasets_with_graphs(dataset_name)
    # графы для обучающей и тестовой выборки
    G_tr = [create_networkx_graph(a, x) for a, x in zip(A_train, X_train)]
    G_te = [create_networkx_graph(a, x) for a, x in zip(A_test, X_test)]

    G_train = list(graph_from_networkx(G_tr, node_labels_tag='features'))
    G_test = list(graph_from_networkx(G_te, node_labels_tag='features'))

    # нужно преобразовать метки классов, тк сейчаас это не одномерный массив
    y_train = np.argmax(y_train, axis=-1)
    y_test = np.argmax(y_test, axis=-1)


    kernels = [
        ShortestPath(normalize=True),
        GraphletSampling(k=5, normalize=True), 
        WeisfeilerLehman(n_iter=5, normalize=True), 
        NeighborhoodHash(random_state=10, normalize=True), 
        # NeighborhoodHash(random_state=12, normalize=True), 
        PyramidMatch(normalize=True), 
        OddSth(normalize=True), 
        CoreFramework(normalize=True),
        Propagation(random_state=12, normalize=True), 
        RandomWalk(normalize=True), 
        SvmTheta(normalize=True), 
        VertexHistogram(normalize=True)
    ]
    results = []

    for gk in kernels:
        
        start = time.time()
        
        K_train = gk.fit_transform(G_train)
        K_test = gk.transform(G_test)
        
        K_train = np.nan_to_num(K_train, nan=0.5)
        K_test = np.nan_to_num(K_test, nan=0.5)
        
        clf_svm = SVC(kernel='precomputed', C=1)
        clf_svm.fit(K_train, y_train)
        y_pred_svm = clf_svm.predict(K_test)
        acc_svm = accuracy_score(y_test, y_pred_svm)
        
        
        y_train_binary = (y_train == 1).astype(int)  # преобразуем метки в бинарные для Ridge
        y_test_binary = (y_test == 1).astype(int)

        clf_ridge = Ridge(alpha=1.0)
        clf_ridge.fit(K_train, y_train_binary)
        y_pred_ridge = clf_ridge.predict(K_test)
        y_pred_ridge_binary = np.clip(y_pred_ridge, 0, 1)  # обрезаем значения меньше 0 и больше 1
        y_pred_ridge_binary = (y_pred_ridge_binary >= 0.5).astype(int) 
        acc_ridge = 0
        acc_ridge = accuracy_score(y_test_binary, y_pred_ridge_binary)
        
        end = time.time()
        results.append({
            "dataset_name": dataset_name, 
            "kernel": gk.__class__.__name__,
            "SVM accuracy": round(acc_svm * 100, 2),
            "Ridge accuracy": round(acc_ridge * 100, 2),
            "time": round(end - start, 2)
        })
        
    
    results_df = pd.DataFrame(results)

    return results_df


In [18]:
easy_small = kernel_trick_("easy_small")

In [19]:
hard_small = kernel_trick_("hard_small")

In [20]:
# easy = kernel_trick_("easy")
# hard = kernel_trick_("hard")

# При выполнении кода в текущей ячейке или предыдущей ячейке ядро аварийно завершило работу. 
# Проверьте код в ячейках, чтобы определить возможную причину сбоя. 
# Щелкните здесь, чтобы получить дополнительные сведения. 
# Подробнее см. в журнале Jupyter.
# "Системе не хватает программной памяти"

# поэтому без этих двух датасетов:(

In [21]:
df = pd.concat([easy_small, hard_small], ignore_index=True)
df

,dataset_name,kernel,SVM accuracy,Ridge accuracy,time
0,easy_small,ShortestPath,100.00,100.00,2.23
1,easy_small,GraphletSampling,41.94,67.74,94.30
2,easy_small,WeisfeilerLehman,51.61,70.97,0.24
3,easy_small,NeighborhoodHash,96.77,90.32,0.84
4,easy_small,PyramidMatch,51.61,70.97,1.01
5,easy_small,OddSth,77.42,93.55,14.17
6,easy_small,CoreFramework,100.00,100.00,16.66
7,easy_small,Propagation,90.32,93.55,0.90
8,easy_small,RandomWalk,19.35,48.39,141.43
9,easy_small,SvmTheta,29.03,70.97,1.44


In [22]:
easy_small

,dataset_name,kernel,SVM accuracy,Ridge accuracy,time
0,easy_small,ShortestPath,100.00,100.00,2.23
1,easy_small,GraphletSampling,41.94,67.74,94.30
2,easy_small,WeisfeilerLehman,51.61,70.97,0.24
3,easy_small,NeighborhoodHash,96.77,90.32,0.84
4,easy_small,PyramidMatch,51.61,70.97,1.01
5,easy_small,OddSth,77.42,93.55,14.17
6,easy_small,CoreFramework,100.00,100.00,16.66
7,easy_small,Propagation,90.32,93.55,0.90
8,easy_small,RandomWalk,19.35,48.39,141.43
9,easy_small,SvmTheta,29.03,70.97,1.44


In [23]:
hard_small 

,dataset_name,kernel,SVM accuracy,Ridge accuracy,time
0,hard_small,ShortestPath,69.23,69.23,1.12
1,hard_small,GraphletSampling,38.46,65.38,13.79
2,hard_small,WeisfeilerLehman,23.08,65.38,0.23
3,hard_small,NeighborhoodHash,76.92,65.38,0.79
4,hard_small,PyramidMatch,23.08,65.38,1.02
5,hard_small,OddSth,42.31,65.38,6.12
6,hard_small,CoreFramework,69.23,69.23,6.01
7,hard_small,Propagation,53.85,69.23,1.41
8,hard_small,RandomWalk,23.08,38.46,146.89
9,hard_small,SvmTheta,15.38,65.38,0.53


In [24]:
# подберем лучшее ядро для всех случаев 
def select_best_kernel(df):
    df['SVM accuracy'] = pd.to_numeric(df['SVM accuracy'])
    df['Ridge accuracy'] = pd.to_numeric(df['Ridge accuracy'])
    
    df = df.pivot_table(index='kernel', columns='dataset_name', values=['SVM accuracy', 'Ridge accuracy', 'time'], aggfunc='first')
    df.columns = [f"{col[0]} {col[1]}" for col in df.columns]
    df.reset_index(inplace=True)
    
    accuracy_columns = df.filter(like='accuracy').columns
    df['Total accuracy'] = df[accuracy_columns].sum(axis=1)
    best_df = df.loc[df['Total accuracy'].idxmax()]

    return best_df


In [25]:
best_kernels_df = select_best_kernel(df)
best_kernels_df['kernel']

'CoreFramework'

## Дополнительно 
 - поэкспериментируйте с kernel construction на основе kernels из библиотеки и попробуйте предложить свой kernel, который работает лучше

Попробуем композиции различных ядер

In [26]:
class SumKernel:
    # преобразуем каждый граф с помощью каждого ядра и суммируем результаты
    def __init__(self, kernels):
        self.kernels = kernels

    def fit_transform(self, graphs):
        kernels_results = [kernel.fit_transform(graphs) for kernel in self.kernels]
        return np.sum(kernels_results, axis=0) 

    def transform(self, graphs):
        kernels_results = [kernel.transform(graphs) for kernel in self.kernels]
        return np.sum(kernels_results, axis=0)
    
class ProductKernel:
    # преобразуем каждый граф с помощью каждого ядра и перемножаем результаты и еще умножаем на константу
    def __init__(self, kernels, const):
        self.kernels = kernels
        self.const = const 

    def fit_transform(self, graphs):
        kernels_results = [kernel.fit_transform(graphs) for kernel in self.kernels]
        return self.const*np.prod(kernels_results, axis=0)  

    def transform(self, graphs):
        kernels_results = [kernel.transform(graphs) for kernel in self.kernels]
        return self.const*np.prod(kernels_results, axis=0)


In [27]:
def custom_kernel_construction(dataset_name):
    # делаем аналогично тому, что выше, но с кастомными ядрами
    A_train, X_train, y_train, A_test, X_test, y_test = load_datasets_with_graphs(dataset_name)


    G_tr = [create_networkx_graph(a, x) for a, x in zip(A_train, X_train)]
    G_te = [create_networkx_graph(a, x) for a, x in zip(A_test, X_test)]

    G_train = list(graph_from_networkx(G_tr, node_labels_tag='features'))
    G_test = list(graph_from_networkx(G_te, node_labels_tag='features'))

    y_train = np.argmax(y_train, axis=-1)
    y_test = np.argmax(y_test, axis=-1)

    kernels = [
        SumKernel([ShortestPath(normalize=True), CoreFramework(normalize=True), NeighborhoodHash(random_state=10, normalize=True)]), 
        ProductKernel([ShortestPath(normalize=True), CoreFramework(normalize=True)], 3)
    ]

    results = []

    for gk in kernels:
        start = time.time()
        
        K_train = gk.fit_transform(G_train)
        K_test = gk.transform(G_test)
        
        K_train = np.nan_to_num(K_train, nan=0.5)
        K_test = np.nan_to_num(K_test, nan=0.5)
        
        clf_svm = SVC(kernel='precomputed', C=1)
        clf_svm.fit(K_train, y_train)
        y_pred_svm = clf_svm.predict(K_test)
        acc_svm = accuracy_score(y_test, y_pred_svm)
        
        y_train_binary = (y_train == 1).astype(int)
        y_test_binary = (y_test == 1).astype(int)

        clf_ridge = Ridge(alpha=1.0)
        clf_ridge.fit(K_train, y_train_binary)
        y_pred_ridge = clf_ridge.predict(K_test)
        y_pred_ridge_binary = np.clip(y_pred_ridge, 0, 1)
        y_pred_ridge_binary = (y_pred_ridge_binary >= 0.5).astype(int)
        acc_ridge = accuracy_score(y_test_binary, y_pred_ridge_binary)
        
        end = time.time()
        results.append({
            "dataset_name": dataset_name,
            "kernel": gk.__class__.__name__,
            "SVM accuracy": round(acc_svm * 100, 2),
            "Ridge accuracy": round(acc_ridge * 100, 2),
            "time": round(end - start, 2)
        })

    results_df = pd.DataFrame(results)
    return results_df


In [28]:
df_custom_k_hard = custom_kernel_construction("hard_small")
df_custom_k_easy = custom_kernel_construction("easy_small")
df_custom_k = pd.concat([df_custom_k_easy, df_custom_k_hard], ignore_index=True)
df_custom_k

,dataset_name,kernel,SVM accuracy,Ridge accuracy,time
0,easy_small,SumKernel,100.00,96.77,19.82
1,easy_small,ProductKernel,100.00,100.00,18.77
2,hard_small,SumKernel,76.92,84.62,7.56
3,hard_small,ProductKernel,80.77,76.92,6.56


- на датасете hard_small получили лучше результаты, чем те, которые были при примение различных ядер (лучшие были 76,92 - svm и 69,23 - ridge reg на разных ядрах, а сейчас 80.77 - svm и 84.62 - ridge reg)
- на easy_small и так получали 100 и сейчас получили 100

# Задание 3.

Подберите несколько датасетов с большим количеством предикторов (>20) из UCI репозитория или любого другого (классификация и/или регрессия). Реализуйте gaussian process (можно взять готовый но скорее всего его все равно придется модфицировать) для подбора гиперпараметров random forest (или какого либо бустинга) (смотрите лекцию 3). Сравните скорость с random search (попытайтесь презойти).

**Дополнительно**: поэксперементируйте с критериями подбора следующей точки (лекция 3) и ядрами, а также с самими критериями качества (есть ли отличие в эффективности gaussian process для разных критериев)

P. S.: в этом задании оцениваться будет прежде всего правильность реализации, а не скорость, т.к. сделать быстрее рандома (в виду требуемых доп вычислений) может быть сложно и это зависит от датасета, но нужно попытаться реализовать как можно эффективнее.

Попробуем несколько датасетов:
- breast cancer (один из самых популярных датасетов uci) c 30 features
- Predict Students' Dropout and Academic Success: 36 features
- Default of Credit Card Clients: 23 features
- Optical Recognition of Handwritten Digits: 64 features
- Connectionist Bench (Sonar, Mines vs. Rocks): 60 features

In [29]:
def load_credit_card_clients():
    default_of_credit_card_clients = fetch_ucirepo(id=350) 
    X = default_of_credit_card_clients.data.features 
    y = default_of_credit_card_clients.data.targets 
    data = SimpleNamespace(data=X, target=y)
    return data

def load_students():
    predict_students_dropout_and_academic_success = fetch_ucirepo(id=697)
    X = predict_students_dropout_and_academic_success.data.features 
    y = predict_students_dropout_and_academic_success.data.targets 
    data = SimpleNamespace(data=X, target=y)
    return data

def load_mushrooms():
    mushroom = fetch_ucirepo(id=73) 
    X = mushroom.data.features 
    for column in X.columns:
        if X[column].dtype == 'object': 
            encoder = LabelEncoder()
            X[column] = encoder.fit_transform(X[column])
    y = mushroom.data.targets 
    y = y['poisonous'].map({'p': 1, 'e': 0})
    data = SimpleNamespace(data=X, target=y)
    return data

def load_digits():
    optical_recognition_of_handwritten_digits = fetch_ucirepo(id=80) 
    X = optical_recognition_of_handwritten_digits.data.features 
    y = optical_recognition_of_handwritten_digits.data.targets 
    data = SimpleNamespace(data=X, target=y)
    return data

def load_connectionist():
    connectionist_bench_sonar_mines_vs_rocks = fetch_ucirepo(id=151)
    X = connectionist_bench_sonar_mines_vs_rocks.data.features 
    y = connectionist_bench_sonar_mines_vs_rocks.data.targets 
    y = y['class'].map({'M': 1, 'R': 0})
    data = SimpleNamespace(data=X, target=y)
    return data
    


In [30]:
def objective_function(params, X_train, X_test, y_train, y_test, random_st=5):
    n_estimators = int(params[0])
    max_depth = int(params[1])
    min_samples_split = int(params[2])
    min_samples_leaf = int(params[3])

    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                random_state=random_st)  # random forest c текущими параметрами
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return -accuracy  # минимизируем функцию (поэтому знак минус)


In [31]:
# Expected Improvement для gp
def expected_improvement(x, gp, y_best):
    # GP для точки x
    mu, sigma = gp.predict([x], return_std=True)
    if sigma == 0:
        return 0
    improvement = mu - y_best
    Z = improvement / sigma
    EI = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    return EI


# ф-я для оптимизации гиперпараметров с помощью gp
def optimize_gp(X_train, X_test, y_train, y_test, random_st=5, num_iter=10):
    # начальные гиперпараметры (случайные)
    param_space = np.array([[10, 50],  # n_estimators от 10 до 50
                            [5, 20],  # max_depth от 5 до 20
                            [2, 20],  # min_samples_split от 2 до 20
                            [1, 20]])  # min_samples_leaf от 1 до 20

    # начальные случайные точки для обучения GP
    X_train_gp = np.array([[random.randint(10, 50),
                            random.randint(5, 20),
                            random.randint(2, 20),
                            random.randint(1, 20)] for _ in range(num_iter)])

    y_train_gp = np.array([objective_function(params, X_train, X_test, y_train, y_test,random_st=random_st) for params in X_train_gp])  # начальные оценки метрик

    kernel = RBF(1.0, (1e-5, 1e1))  # kernel для GP
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=random_st)
    gp.fit(X_train_gp, y_train_gp)

    # оптимизация гиперпараметров
    for _ in range(5):
        # выбор след точки для сэмплирования с помощью EI
        next_params = None
        max_ei = -np.inf

        for _ in range(50):  # проверяем 50 точек
            candidate = np.random.uniform(param_space[:, 0], param_space[:, 1])
            ei = expected_improvement(candidate, gp, np.min(y_train_gp))
            if ei > max_ei:
                max_ei = ei
                next_params = candidate

        y_new = objective_function(next_params, X_train, X_test, y_train, y_test, random_st=random_st)
        X_train_gp = np.vstack([X_train_gp, next_params])
        y_train_gp = np.append(y_train_gp, y_new)

        gp.fit(X_train_gp, y_train_gp)

    best_params = X_train_gp[np.argmin(y_train_gp)]  # гиперпараметры с минимальной ошибкой
    best_acc = -np.min(y_train_gp)
    return best_params, best_acc

In [32]:
# Random Search
def random_search(X_train, X_test, y_train, y_test, random_st=5, num_iter=10):
    best_acc = -np.inf
    best_params = None
    for _ in range(num_iter):
        params = [random.randint(10, 50),
                  random.randint(5, 20),
                  random.randint(2, 20),
                  random.randint(1, 20)]

        accuracy = -objective_function(params, X_train, X_test, y_train, y_test, random_st=random_st)  # минимизируем 

        if accuracy > best_acc:
            best_acc = accuracy
            best_params = params

    return best_params, best_acc

In [33]:
datasets = {
    'breast_cancer': load_breast_cancer(),
    'students': load_students(),
    "credit_card_clients": load_credit_card_clients(),
    # "mushrooms": load_mushrooms(),
    'digits': load_digits(),
    'connectionist': load_connectionist()

}

def compare_gp_randomsearch(random_st=5):
    results = []
    time_results = []
    
    for dataset_name, dataset in datasets.items():
        X = dataset.data
        y = dataset.target

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_st)

        # Gaussian Process
        start_time = time.time()
        best_params_gp, best_acc_gp = optimize_gp(X_train, X_test, y_train, y_test, random_st=random_st)
        gp_time = time.time() - start_time

        # Random Search
        start_time = time.time()
        best_params_random, best_acc_random = random_search(X_train, X_test, y_train, y_test, random_st=random_st)
        random_time = time.time() - start_time
        
        results.append({
            "Dataset": dataset_name,
            "Best GP params": [f"{best_param_gp:.2f}" for best_param_gp in best_params_gp],
            "Best GP accuracy": best_acc_gp,
            "GP time": gp_time,
            "Best Random Search params": best_params_random,
            "Best Random Search accuracy": best_acc_random,
            "Random Search time": random_time,
            
        })
        
    return pd.DataFrame(results)
        
compare_gp_randomsearch()


,Dataset,Best GP params,Best GP accuracy,GP time,Best Random Search params,Best Random Search accuracy,Random Search time
0,breast_cancer,"[25.00, 15.00, 3.00, 7.00]",0.973684,0.432448,"[16, 19, 15, 15]",0.973684,0.241130
1,students,"[42.04, 18.48, 3.47, 5.81]",0.811299,1.169018,"[19, 8, 2, 8]",0.803390,0.676129
2,credit_card_clients,"[47.52, 19.50, 20.00, 15.46]",0.821833,12.389363,"[37, 15, 20, 18]",0.821500,9.298777
3,digits,"[46.00, 19.00, 4.00, 3.00]",0.976868,1.821994,"[28, 10, 6, 3]",0.971530,1.009588
4,connectionist,"[47.89, 12.76, 3.43, 1.35]",0.880952,0.301829,"[33, 12, 10, 2]",0.833333,0.144626


**Как видно, gp для подбора гиперпараметров работает дольше на 5/5 датасетов. Причем дольше в 1,5-2 раза (но как было описано в условии задания - это ожидаемый результат)**


Чтобы сравнить accuracy двух методов, попробуем их применить при разных random_state и посмотреть в скольких случаях gp оказался лучше RS.

In [34]:
def compare_gp_rs_ntimes(n_iter=10):
    all_results = [] 

    for _ in range(n_iter):
        random_st = random.randint(1, 100)
        df = compare_gp_randomsearch(random_st=random_st) 

        new_data = []
        
        for _, row in df.iterrows():
            dataset = row['Dataset']
            best_gp_accuracy = row['Best GP accuracy']
            best_rs_accuracy = row['Best Random Search accuracy']
            
            if best_gp_accuracy > best_rs_accuracy:  # gp лучше, чем rs
                gp_better = 1
                gp_eq_rs = 0
                rs_better_gp = 0
            elif best_gp_accuracy < best_rs_accuracy:  # gp хуже, чем rs
                gp_better = 0
                gp_eq_rs = 0
                rs_better_gp = 1
            else:  # значения равны
                gp_better = 0
                gp_eq_rs = 1
                rs_better_gp = 0

            new_data.append([dataset, gp_better, gp_eq_rs, rs_better_gp])

        new_df = pd.DataFrame(new_data, columns=['Dataset', 'GP better than RS', 'GP acc = RS acc', 'RS better than GP'])
        all_results.append(new_df)
        
    all_results_df = pd.concat(all_results, ignore_index=True)
    
    return all_results_df.groupby('Dataset', as_index=False).sum()


In [35]:
compare_gp_rs_ntimes()

,Dataset,GP better than RS,GP acc = RS acc,RS better than GP
0,breast_cancer,5,4,1
1,connectionist,8,1,1
2,credit_card_clients,7,0,3
3,digits,7,1,2
4,students,9,1,0


Можно заметить, что в большинстве случаев gp демонстрирует лучшие результаты, чем rs (хоть gp и дольше), то есть на каждом датасете число случаев, когда accuracy у gp оказалась больше, чем у rs, сильно больше, чем число случаев, когда accuracy у rs оказалась больше, чем у gp

## Дополнительно
- поэксперементируйте с критериями подбора следующей точки (лекция 3) и ядрами, а также с самими критериями качества (есть ли отличие в эффективности gaussian process для разных критериев)

Попробуем добавить Probability of Improvement (PI), Upper Confidence Bound (UCB) из 3 лекции и три ядра RBF, Matern и Constant (его комбинацию с Matern и RBF)

In [36]:
def probability_of_improvement(x, gp, y_best):
    mu, sigma = gp.predict([x], return_std=True)
    Z = (mu - y_best) / sigma
    PI = norm.cdf(Z)
    return PI

def upper_confidence_bound(x, gp, k=2.0):
    mu, sigma = gp.predict([x], return_std=True)
    UCB = mu + k * sigma
    return UCB

In [37]:
kernels_d = {
    'RBF':  RBF(1.0, (1e-5, 1e1)),
    'Matern': Matern(length_scale=1.0, nu=2.5, length_scale_bounds=(1e-5, 1e1)),
    'ConstRBF': ConstantKernel(2.0, (1e-5, 1e1)) + RBF(1.0, (1e-5, 1e1)),
    'ConstMatern': ConstantKernel(2.0, (1e-5, 1e1)) + Matern(length_scale=1.0, nu=2.5, length_scale_bounds=(1e-5, 1e1)),
}

In [38]:
def optimize_gp_additional(X_train, X_test, y_train, y_test, kernel_type='RBF', ac_function='EI', random_st=5, num_iter=10):
    param_space = np.array([[10, 50], [5, 20], [2, 20], [1, 20]])
    
    # начальные случайные точки для обучения GP
    X_train_gp = np.array([[random.randint(10, 50),
                            random.randint(5, 20),
                            random.randint(2, 20),
                            random.randint(1, 20)] for _ in range(num_iter)])

    y_train_gp = np.array([objective_function(params, X_train, X_test, y_train, y_test,random_st=random_st) for params in X_train_gp])
    
    kernel = kernels_d[kernel_type]
    
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=random_st)
    gp.fit(X_train_gp, y_train_gp)

    for _ in range(5):
        next_params = None
        max_ac = -np.inf

        for _ in range(50): 
            candidate = np.random.uniform(param_space[:, 0], param_space[:, 1])

            if ac_function == 'EI':
                ac_value = expected_improvement(candidate, gp, np.min(y_train_gp))
            elif ac_function == 'PI':
                ac_value = probability_of_improvement(candidate, gp, np.min(y_train_gp))
            elif ac_function == 'UCB':
                ac_value = upper_confidence_bound(candidate, gp)

            if ac_value > max_ac:
                max_ac = ac_value
                next_params = candidate

        y_new = objective_function(next_params, X_train, X_test, y_train, y_test, random_st=random_st)
        X_train_gp = np.vstack([X_train_gp, next_params])
        y_train_gp = np.append(y_train_gp, y_new)

        gp.fit(X_train_gp, y_train_gp)

    best_params = X_train_gp[np.argmin(y_train_gp)] 
    best_acc = -np.min(y_train_gp)
    return best_params, best_acc

In [43]:
datasets = {
    'breast_cancer': load_breast_cancer(),
    'students': load_students(),
    # "credit_card_clients": load_credit_card_clients(), # очень долгий
    # "mushrooms": load_mushrooms(),
    'digits': load_digits(),
    'connectionist': load_connectionist()

}
# GP и Random Search c разными ядрами
def compare_gp_randomsearch_additional(random_st=5):
    results = []
    
    for dataset_name, dataset in datasets.items():
        X = dataset.data
        y = dataset.target

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_st)

        # Gaussian Process
        best_kernel = None
        best_ac_fun = None
        best_acc_gp = 0
        res_time_gp = 0
        for kernel_type, _ in kernels_d.items():
            for f in ['EI', 'UCB', 'PI']:
                start_time = time.time()
                best_params_gp, acc_gp = optimize_gp_additional(X_train, X_test, y_train, y_test, kernel_type=kernel_type, ac_function=f, random_st=random_st)
                gp_time = time.time() - start_time
                
                if acc_gp > best_acc_gp:
                    best_acc_gp = acc_gp
                    best_kernel = kernel_type
                    best_ac_fun = f
                    res_time_gp = gp_time
                    

        # Random Search
        start_time = time.time()
        best_params_random, best_acc_random = random_search(X_train, X_test, y_train, y_test, random_st=random_st)
        random_time = time.time() - start_time
        
        results.append({
            "Dataset": dataset_name,
            "Best GP accuracy": best_acc_gp,
            "Best GP kernel": best_kernel,
            "Best GP aquisition function": best_ac_fun,
            "GP time": res_time_gp,
            "Best Random Search accuracy": best_acc_random,
            "Random Search time": random_time,
        })
        
    return pd.DataFrame(results)

compare_gp_randomsearch_additional()

,Dataset,Best GP accuracy,Best GP kernel,Best GP aquisition function,GP time,Best Random Search accuracy,Random Search time
0,breast_cancer,0.982456,RBF,EI,0.588609,0.964912,0.383906
1,students,0.805650,RBF,EI,2.211132,0.801130,1.213934
2,digits,0.979537,ConstRBF,EI,3.045867,0.975089,2.117088
3,connectionist,0.880952,RBF,UCB,0.779022,0.833333,0.276734


 Теперь можно увидеть, что на всех датасетах GP показывает лучшую accuracy, чем RS (но на несколько сотых, а где-то  даже тысячных )